In [1]:
# script pion.py hjyf
from tkinter import *
import random

import numpy as np

from mdpmadi22_class import Display

In [2]:
nblignes = 10
nbcolonnes = 20

display = Display(2, 0, nblignes, nbcolonnes, 0)
display.initialize()

In [3]:
class state:
    def __init__(self, coords, Value, reward, dictNStates, dictAroundStates, isReward):
        self.coords = coords
        self.value = Value
        self.reward = reward
        self.bestAction = ' '
        self.IsaReward = isReward
        self.dictNextStates = dictNStates
        self.nbNextStates = len(self.dictNextStates)
        self.dictAroundStates = dictAroundStates
        self.nbStatesAround = len(dictAroundStates)
        self.Probability = 1 - self.nbStatesAround/16 


    def __repr__(self) -> str:
        return f"coord: {self.coords}\nvalue: {self.value}\nreward: {self.reward}\ndictNStates: {self.dictNextStates}\ndictAroundStates: {self.dictAroundStates}\nnbStatesAround: {self.nbStatesAround}"

In [4]:
class env:
    def __init__(self, nbcolonnes, nblignes, g, weight):
        self.nbcolonnes = nbcolonnes
        self.nblignes = nblignes
        self.g = g
        self.weight = weight
        self.listAllStates = self.create_all_states()

    def create_all_states(self):
        l = list()
        for i in range(self.nblignes):
            for j in range(self.nbcolonnes):
                coord = (i, j)
                isaReward = False
                if self.g[i, j] < 0:
                    reward = ' mur '
                else:
                    # reward = np.format_float_scientific(self.weight[g[i,j]],exp_digits=1)
                    reward = -self.weight[self.g[i, j]]
                    if reward > 0:
                        isaReward = True
                value = 0
                dictNStates = self.get_dictNextStates(i, j)
                dictAroundStates = self.get_around_States_Proba(i, j)

                s = state(
                    coord,
                    value,
                    reward,
                    dictNStates,
                    dictAroundStates,
                    isaReward,
                )
                l.append(s)
        return l

    def get_dictNextStates(self, i, j):
        # check OutOfBound
        di = dict()
        if i - 2 >= 0 and j + 1 < self.nbcolonnes and self.g[i - 2, j + 1] != -1:
            di['y'] = (i - 2, j + 1)
        if i - 1 >= 0 and j + 2 < self.nbcolonnes and self.g[i - 1, j + 2] != -1:
            di['u'] = (i - 1, j + 2)
        if i + 1 < self.nblignes and j + 2 < self.nbcolonnes and self.g[i + 1, j + 2] != -1:
            di['j'] = (i + 1, j + 2)
        if i + 2 < self.nblignes and j + 1 < self.nbcolonnes and self.g[i + 2, j + 1] != -1:
            di['h'] = (i + 2, j + 1)
        if i + 2 < self.nblignes and j - 1 >= 0 and self.g[i + 2, j - 1] != -1:
            di['g'] = (i + 2, j - 1)
        if i + 1 < self.nblignes and j - 2 >= 0 and self.g[i + 1, j - 2] != -1:
            di['f'] = (i + 1, j - 2)
        if i - 1 >= 0 and j - 2 >= 0 and self.g[i - 1, j - 2] != -1:
            di['r'] = (i - 1, j - 2)
        if i - 2 >= 0 and j - 1 >= 0 and self.g[i - 2, j - 1] != -1:
            di['t'] = (i - 2, j - 1)

        return di

    def get_around_States_Proba(self, i, j):
        # check OutOfBound
        diStateAround = dict()
        if i + 1 < self.nblignes and j < self.nbcolonnes and self.g[i
                + 1, j] != -1:
            diStateAround['md'] = (i + 1, j, 1 / 16)

        if i - 1 >= 0 and j < self.nbcolonnes and self.g[i - 1, j] != -1:
            diStateAround['mg'] = (i - 1, j, 1 / 16)

        if i + 1 < self.nblignes and j + 1 < self.nbcolonnes and self.g[i + 1, j + 1] != -1:
            diStateAround['hd'] = (i + 1, j + 1, 1 / 16)

        if i - 1 >= 0 and j - 1 >= 0 and self.g[i - 1, j - 1] != -1:
            diStateAround['bg'] = (i - 1, j - 1, 1 / 16)

        if i + 1 < self.nblignes and j - 1 >= 0 and self.g[i + 1, j
                - 1] != -1:
            diStateAround['bd'] = (i + 1, j - 1, 1 / 16)

        if i - 1 >= 0 and j + 1 < self.nbcolonnes and self.g[i - 1, j
                + 1] != -1:
            diStateAround['hg'] = (i - 1, j + 1, 1 / 16)

        if i < self.nblignes and j + 1 < self.nbcolonnes and self.g[i,
                j + 1] != -1:
            diStateAround['hm'] = (i, j + 1, 1 / 16)

        if i < self.nblignes and j - 1 >= 0 and self.g[i, j - 1] != -1:
            diStateAround['bm'] = (i, j - 1, 1 / 16)

        return diStateAround

    def get_state(self, i, j):
        i = int(i)
        j = int(j)
        if i >= 0 and i < self.nblignes and j >= 0 and j < self.nbcolonnes:
            return self.listAllStates[i * self.nbcolonnes + j]
        return None

    def afficheV(self):
        print("Valeur de l'etat selon l'algo")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                value = '{:.2e}'.format(self.get_state(i, j).value)
                l.append(value)
            print(l)

    def afficheBestAction(self):
        print('meilleure Action')
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i, j).bestAction)
            print(l)

    def afficheP(self):
        print("Proba d'arriver ce State")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i, j).Probability)
            print(l)

    def afficheNBSA(self):
        print('NB State Around')
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i, j).nbStatesAround)
            print(l)

    def afficheR(self):
        print('Rewards')
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i, j).reward)
            print(l)

    def afficheNBNS(self):
        print('NB Next State ')
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append((self.get_state(i, j).nbNextStates,
                         self.get_state(i, j).dictNextStates))
            print(l)


In [5]:
e = env(nbcolonnes, nblignes, display.g, weight=display.weight)
    
e.afficheV()
e.afficheP()
e.afficheNBSA()
e.afficheR()
e.afficheBestAction()
e.afficheNBNS() 

Valeur de l'etat selon l'algo
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00

In [6]:
def value_iterationG(nbcolonnes,nblignes, gamma, epsilon):
    Value_states = np.zeros(nblignes*nbcolonnes, dtype=int)
    e = env(nbcolonnes,nblignes,display.g,weight=display.weight)
    
    e.afficheV()
    e.afficheP()
    e.afficheP()
    # actions = ['f','g','h','j','y','u',]
    iteration = 0
    stop = False

    scores = []

    while not stop:
        
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward:
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                somme = 0
                pState = e.get_state(v[0],v[1]) 
                somme = pState.Probability*pState.value 
                #print(pState.Probability)             
                for cState in pState.dictAroundStates.values():
                    
                    sA = e.get_state(cState[0],cState[1])
                    somme += (1/16*sA.value)
                        
                temp = s.reward+gamma*somme

                if temp > best:
                    best = temp
                    saveA = a
                    # print(temp)
                
            s.bestAction = saveA
            s.value = best
            # print(tempBest)
            # for i in tempBest :
            #     bestV = 0
            #     bestA = 'no'
            #     if i[1]>bestV :
            #         bestV = i[1]
            #         bestA = i[0]

            # s.value = bestV
            # s.bestAction = bestA
            
          
            
            
        iteration +=1
       

        if iteration>1000:
            stop = True
            e.afficheV()
            e.afficheBestAction()
            e.afficheR()
            print(iteration)
            for i in range(0, len(scores), 10):
                print(scores[i])
    

gamma = 0.9
epsilon = 1

value_iterationG(nbcolonnes=nbcolonnes,nblignes=nblignes, gamma=gamma, epsilon=epsilon)
    

Valeur de l'etat selon l'algo
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00

In [7]:
display.run()



: 